In [10]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import PromptTemplate
from langchain.agents import initialize_agent
from langchain.chains import LLMChain
from IPython.display import Image, display
from typing_extensions import TypedDict, Annotated
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import create_react_agent

In [2]:
model = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0.05,
    timeout=None,
    max_retries=2,
)

In [4]:
template = """
You are an assistant for improving CVs for a given job description.
Use the following job description and CV to suggest improvements.
Do not make up any information, only use the information provided.
Try not to overuse words like "spearheaded", "expert", "innovative", but do not be afraid to suggest changes.
Quantify where possible, but do not make up numbers.
If you think key skills are missing, suggest what they should be.
If you think the CV is missing information, suggest what it should be.
If you think the CV is not written in a professional tone, suggest how it should be improved.
Keep the answer concise and in British English.

{job_description}

{cv}

Suggested improvements to the CV:
"""

custom_prompt = PromptTemplate.from_template(template)

In [9]:
class CVReviewerAgent(TypedDict):
    """CV Reviewer Agent."""
    job_description: Annotated[str, ..., "The job description to review the CV against"]
    cv: Annotated[str, ..., "The CV to review and suggest improvements for"]

In [36]:
# Step 2: Rewrite sections of the CV based on the suggestions
rewrite_template = """
Given the following CV and the suggested improvements, rewrite the relevant sections of the CV incorporating the suggestions.
Only rewrite the sections that need improvement based on the suggestions. Keep the rest of the CV unchanged.
If you are unsure if the person has the skill, leave it unchanged.
Do not make up information, only use the information provided.
If you are unsure of what to rewrite, leave it unchanged.

CV:
{cv}

Suggested improvements:
{suggestions}

Rewritten CV sections:
"""

rewrite_prompt = PromptTemplate.from_template(rewrite_template)
rewrite_chain = LLMChain(llm=model, prompt=rewrite_prompt)

In [37]:
def run(job_description: str, cv: str) -> str:
    llm_chain = LLMChain(llm=model, prompt=custom_prompt)

    # Remove the tools and agent initialization since no tools are being used
    result = llm_chain.invoke(input={
        "job_description": job_description,
        "cv": cv
    })

    suggestions = result["text"]
    print("Suggested improvements to the CV:")
    print()
    for line in suggestions.split("\n"):
        print(f"{line}")

    rewrite_result = rewrite_chain.invoke(input={
        "cv": cv,
        "suggestions": suggestions
    })

    rewritten_sections = rewrite_result["text"]
    print("\nRewritten CV sections:")
    print(rewritten_sections)

    return suggestions, rewritten_sections

In [27]:
with open("tmp/cv.txt", "r") as file:
    cv = file.read()

In [28]:
with open("tmp/meta.txt", "r") as file:
    job_description = file.read()

In [38]:
r = run(job_description, cv)

Suggested improvements to the CV:

Based on the job description, here are the key suggested improvements for the CV:

1. Summary Section:
- Add specific mention of recommendation systems and classification experience, as these are key requirements
- Include experience with large-scale systems, as Meta emphasizes scale
- Mention leadership experience more prominently

2. Work Experience:
- Quantify team sizes led/influenced where possible
- Add more details about recommendation systems work if any exists
- Highlight any experience with distributed systems and GPU optimization
- Add examples of technical leadership and cross-team collaboration

3. Skills Section:
- Add C/C++ and Java if you have experience with them
- Include TensorFlow alongside PyTorch as it's preferred
- Add any experience with distributed computing platforms
- Consider reorganizing to emphasize ML systems at scale

Missing Elements to Consider Adding:
- Any experience with recommendation systems
- Examples of setting